In [4]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
labeled_data = pd.read_csv('../../Notebook/data/labeled_data.csv')
labels = labeled_data.iloc[:, 2:].values   # [xmin, ymin, xmax, ymax]

In [33]:
dirs = r'E:\YOLO-ALPD\Notebook\data\images_labeled'
# len(labels)  # 225

In [38]:
images_array = []
labels_array = []

for index in range(len(labels)):
    # load the images
    image_path = dirs + "\\" + labeled_data.iloc[index]['filename']
    img_arr = cv2.imread(image_path)
    height, width, dim=img_arr.shape
    height, width, dim

    # Here we split the dataset
    load_image = load_img(image_path, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    normalize_image_arr = load_image_arr / 255.0

    # normalize the labels
    xmin, ymin, xmax, ymax = labels[index]
    xmin, xmax = xmin/width, xmax/width
    ymin, ymax = ymin/height, ymax/height
    label_noms = (xmin, xmax, ymin, ymax)
    # label_noms
    
    # append the array
    images_array.append(normalize_image_arr)
    labels_array.append(label_noms)
    

In [ ]:
len(images_array)
len(labels_array)
np.array(images_array)

array([[[[0.8352941 , 0.9764706 , 0.96862745],
         [0.8392157 , 1.        , 0.99607843],
         [0.8392157 , 1.        , 0.99607843],
         ...,
         [0.5764706 , 0.44313726, 0.4392157 ],
         [0.57254905, 0.4509804 , 0.44313726],
         [0.64705884, 0.5254902 , 0.5176471 ]],

        [[0.9137255 , 1.        , 1.        ],
         [0.88235295, 1.        , 1.        ],
         [0.87058824, 0.99607843, 0.99215686],
         ...,
         [0.5921569 , 0.4392157 , 0.44313726],
         [0.52156866, 0.3882353 , 0.39215687],
         [0.54901963, 0.41568628, 0.41960785]],

        [[0.89411765, 0.99215686, 0.9764706 ],
         [0.8980392 , 1.        , 0.99607843],
         [0.88235295, 1.        , 1.        ],
         ...,
         [0.5529412 , 0.39607844, 0.4       ],
         [0.5647059 , 0.4117647 , 0.42352942],
         [0.5411765 , 0.4       , 0.40784314]],

        ...,

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 0.99215686]

In [48]:
# split the data 
X = np.array(images_array)
y = np.array(labels_array)

X.shape, y.shape

((225, 224, 224, 3), (225, 4))

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# Model Training

In [61]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Input, Flatten
from tensorflow.keras.models import Model


In [ ]:
'''
keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    name="inception_resnet_v2",
)
'''

# load the pretrained model
inception_restnet = InceptionResNetV2(
    include_top = False,
    weights="imagenet",
    input_tensor=Input(shape=(224,224,3)), 
)

inception_restnet.trainable=False   # don't want to chnages of imagenet pretrained weight

headmodel = inception_restnet.output
headmodel = Flatten()(headmodel)  # output features extractions from this model
# start building the nurons
headmodel = Dense(500, activation='relu')(headmodel)
headmodel = Dense(250, activation='relu')(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)   # output of my model

# create new model or tyransfer learning
model = Model(inputs=inception_restnet.input, outputs=headmodel)

In [64]:
# compile the model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1015         │ (None, 111, 111,  │        864 │ input_layer_5[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_1015[0][0] │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1015     │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1016         │ (None, 109, 109,  │      9,216 │ activation_1015[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1016[0][0] │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1016     │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1017         │ (None, 109, 109,  │     18,432 │ activation_1016[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_1017[0][0] │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1017     │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_20    │ (None, 54, 54,    │          0 │ activation_1017[… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1018         │ (None, 54, 54,    │      5,120 │ max_pooling2d_20… │
│ (Conv2D)            │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_1018[0][0] │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1018     │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1019         │ (None, 52, 52,    │    138,240 │ activation_1018[… │
│ (Conv2D)            │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_1019[0][0] │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1019     │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

In [ ]:
# train our model